In [17]:
from sys import platform
import warnings
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd

if platform == "win32":
    path = 'C:/Users/olive/GitHub/F1-Fantasy-Predictor/'
elif platform == "darwin":
    path = '~/Documents/GitHub/F1-Fantasy-Predictor/'

warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [18]:
# Enable the cache
ff1.Cache.enable_cache(path+'data/cache') 
# ff1.Cache.disabled()

In [19]:
df = pd.DataFrame(columns=['season','round','driver', 'time', 'lapNum', 's1', 's2', 's3', 'spI1', 'spI2', 'SpeedFL',
       'spst', 'isPB', 'compound', 'tyrelife', 'isFresh', 'Stint',
       'trackStatus', 'isAccurate'])

In [20]:
year = 2021
df = pd.DataFrame()
for circuit in range(1,23):
    # Load the session data
    try:
        session = ff1.get_session(year, circuit, 'FP3')
        session.load()
        laps = session.load_laps(with_telemetry=True)
        laps = laps[[
        'DriverNumber', 'LapTime', 'LapNumber',
        'Sector1Time', 'Sector2Time', 'Sector3Time',
        'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
        'Compound', 'TyreLife', 'FreshTyre', 'Stint',
        'TrackStatus', 'IsAccurate'
        ]]

        laps = laps.rename(columns={'DriverNumber': 'driver','LapTime': 'time','LapNumber': 'lapNum','Sector1Time': 's1','Sector2Time': 's2','Sector3Time': 's3','SpeedI1': 'spI1','SpeedI2': 'spI2','SpeedST': 'spst', 'IsPersonalBest': 'isPB','Compound': 'compound','TyreLife': 'tyrelife','FreshTyre': 'isFresh','TrackStatus': 'trackStatus','IsAccurate': 'isAccurate',})

        laps = laps[laps['time'].notnull()]

        laps['time'] = laps['time'].apply(lambda x: x.total_seconds())
        laps['s1'] = laps['s1'].apply(lambda x: x.total_seconds())
        laps['s2'] = laps['s2'].apply(lambda x: x.total_seconds())
        laps['s3'] = laps['s3'].apply(lambda x: x.total_seconds())

        fastest_laps = laps.query('isPB == True').sort_values('time')
        fastest_laps['season'] = year
        fastest_laps['round'] = circuit

        df = pd.concat([df, fastest_laps])
    except:
        pass

core           INFO 	Loading data for Bahrain Grand Prix - Practice 3 [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Library/Frameworks/Python

In [21]:
df.shape

(358, 19)

In [33]:
df

,driver,time,lapNum,s1,s2,s3,spI1,spI2,SpeedFL,spst,isPB,compound,tyrelife,isFresh,Stint,trackStatus,isAccurate,season,round
77,33,90.577,8,28.648,39.201,22.728,244.0,268.0,286.0,317.0,True,SOFT,2.0,True,4,1,True,2021,1
63,44,91.316,8,28.722,39.801,22.793,239.0,264.0,290.0,320.0,True,SOFT,2.0,True,3,1,True,2021,1
171,10,91.583,9,28.951,39.680,22.952,242.0,270.0,289.0,321.0,True,SOFT,2.0,True,3,1,True,2021,1
257,77,91.855,12,28.982,39.931,22.942,235.0,260.0,289.0,320.0,True,SOFT,2.0,True,3,1,True,2021,1
160,11,91.908,9,29.106,40.010,22.792,242.0,267.0,288.0,317.0,True,SOFT,2.0,True,3,1,True,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,5,85.115,17,17.450,36.530,31.135,285.0,314.0,222.0,321.0,True,SOFT,2.0,True,4,1,True,2021,22
27,63,85.220,12,17.348,36.743,31.129,288.0,315.0,218.0,325.0,True,SOFT,4.0,True,3,1,True,2021,22
318,6,85.322,10,17.354,36.801,31.167,287.0,315.0,221.0,324.0,True,SOFT,2.0,True,3,1,True,2021,22
70,47,85.340,13,17.464,36.705,31.171,285.0,313.0,215.0,322.0,True,SOFT,4.0,True,3,1,True,2021,22


In [35]:
df.query('compound == "SOFT" & tyrelife <= 3.0').groupby(['round','driver']).agg({
    'time': 'mean'
}).sort_values('time')

time
round driver         
8     44       64.369
      33       64.573
9     33       64.591
8     77       64.832
9     77       65.129
...               ...
6     44      102.697
      3       103.557
      6       104.054
      63      104.434
      47      105.282

[161 rows x 1 columns]

In [37]:
df.to_csv(path+'data/fp3.csv', index=False)